Tensores
=======

Tensores são estruturas de dados especializadas muito semelhantes a arrays e matrizes.  
No PyTorch, usamos tensores para codificar as entradas e saídas de um modelo, bem como os parâmetros do modelo.

Os tensores são semelhantes aos ndarrays [NumPy's](https://numpy.org/), 
exceto pelo fato de que podem ser executados em GPUs ou outros aceleradores de hardware. 
Na verdade, tensores e arrays NumPy podem frequentemente compartilhar a mesma memória subjacente, 
eliminando a necessidade de copiar dados consulte
[Ponte com NumPy](https://docs.pytorch.org/tutorials/beginner/blitz/tensor_tutorial.html#bridge-to-np-label).  
Os tensores também são otimizados para diferenciação automática (veremos mais sobre isso posteriormente na seção 
[Autograd](https://docs.pytorch.org/tutorials/beginner/basics/autogradqs_tutorial.html) section).  
Se você conhece ndarrays, se sentirá em casa com a API do Tensor. Se não, acompanhe!

In [33]:
import torch
import numpy as np

Inicializando um Tensor
=====================

Os tensores podem ser inicializados de várias maneiras. Veja os exemplos a seguir:

**Diretamente dos dados**

Tensores podem ser criados diretamente a partir de dados. O tipo de dado é inferido automaticamente.


In [34]:
data = [[1, 2],[3, 4]]
x_data = torch.tensor(data)

**De uma matriz NumPy**

Tensores podem ser criados a partir de matrizes NumPy e vice-versa - veja
[Ponte com NumPy](https://docs.pytorch.org/tutorials/beginner/blitz/tensor_tutorial.html#bridge-to-np-label).


In [35]:
np_array = np.array(data)
x_np = torch.from_numpy(np_array)

**De outro tensor:**

O novo tensor retém as propriedades (formato, tipo de dados) do tensor do argumento, a menos que seja explicitamente substituído.

In [36]:
x_ones = torch.ones_like(x_data) # retains the properties of x_data
print(f"Ones Tensor: \n {x_ones} \n")

x_rand = torch.rand_like(x_data, dtype=torch.float) # overrides the datatype of x_data
print(f"Random Tensor: \n {x_rand} \n")

Ones Tensor: 
 tensor([[1, 1],
        [1, 1]]) 

Random Tensor: 
 tensor([[0.7272, 0.7436],
        [0.3447, 0.9935]]) 



**Com valores aleatórios ou constantes:**

`shape` é uma tupla de dimensões do tensor. Nas funções abaixo, ela determina a dimensionalidade do tensor de saída.

In [37]:
shape = (2,3,)
rand_tensor = torch.rand(shape)
ones_tensor = torch.ones(shape)
zeros_tensor = torch.zeros(shape)

print(f"Random Tensor: \n {rand_tensor} \n")
print(f"Ones Tensor: \n {ones_tensor} \n")
print(f"Zeros Tensor: \n {zeros_tensor}")

Random Tensor: 
 tensor([[0.5021, 0.2569, 0.7706],
        [0.3851, 0.2296, 0.1285]]) 

Ones Tensor: 
 tensor([[1., 1., 1.],
        [1., 1., 1.]]) 

Zeros Tensor: 
 tensor([[0., 0., 0.],
        [0., 0., 0.]])


------------------------------------------------------------------------


Atributos de um Tensor
======================

Os atributos do tensor descrevem sua forma, tipo de dados e o dispositivo no qual são armazenados.

In [38]:
tensor = torch.rand(3,4)

print(f"Shape of tensor: {tensor.shape}")
print(f"Datatype of tensor: {tensor.dtype}")
print(f"Device tensor is stored on: {tensor.device}")

Shape of tensor: torch.Size([3, 4])
Datatype of tensor: torch.float32
Device tensor is stored on: cpu


------------------------------------------------------------------------


Operações em Tensores
=====================

Mais de 1200 operações tensoriais, incluindo aritmética, álgebra linear, 
manipulação de matrizes (transposição, indexação, fatiamento), 
amostragem e muito mais são descritas de forma abrangente
[aqui](https://pytorch.org/docs/stable/torch.html).

Cada uma dessas operações pode ser executada na CPU e em um
[acelerador](https://pytorch.org/docs/stable/torch.html#accelerators)
como CUDA, MPS, MTIA ou XPU. Se estiver usando o Colab, 
aloque um acelerador acessando Tempo de Execução \> Alterar tipo de tempo de execução \> GPU.

Por padrão, os tensores são criados na CPU. Precisamos mover explicitamente os tensores para o acelerador usando o método `.to` (após verificar a disponibilidade do acelerador). Lembre-se de que copiar tensores grandes entre dispositivos pode ser custoso em termos de tempo e memória!

In [39]:
# We move our tensor to the current accelerator if available
if torch.accelerator.is_available():
    tensor = tensor.to(torch.accelerator.current_accelerator())

Experimente algumas das operações da lista. Se você conhece a API NumPy, verá que usar a API Tensor é muito fácil.


**Indexação e fatiamento padrão do tipo numpy:**


In [40]:
tensor = torch.ones(4, 4)
print(f"First row: {tensor[0]}")
print(f"First column: {tensor[:, 0]}")
print(f"Last column: {tensor[..., -1]}")
tensor[:,1] = 0
print(tensor)

First row: tensor([1., 1., 1., 1.])
First column: tensor([1., 1., 1., 1.])
Last column: tensor([1., 1., 1., 1.])
tensor([[1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.]])


**Unindo tensores** Você pode usar `torch.cat` concatenar uma 
sequência de tensores ao longo de uma dada dimensão. Veja também
[torch.stack](https://pytorch.org/docs/stable/generated/torch.stack.html),
outro operador de junção de tensores que é sutilmente diferente de
`torch.cat`.

In [41]:
t1 = torch.cat([tensor, tensor, tensor], dim=1)
print(t1)

tensor([[1., 0., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1.],
        [1., 0., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1.],
        [1., 0., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1.],
        [1., 0., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1.]])


**Operações aritméticas**


In [42]:
# This computes the matrix multiplication between two tensors. y1, y2, y3 will have the same value
# ``tensor.T`` returns the transpose of a tensor
y1 = tensor @ tensor.T
y2 = tensor.matmul(tensor.T)

y3 = torch.rand_like(y1)
torch.matmul(tensor, tensor.T, out=y3)


# This computes the element-wise product. z1, z2, z3 will have the same value
z1 = tensor * tensor
z2 = tensor.mul(tensor)

z3 = torch.rand_like(tensor)
torch.mul(tensor, tensor, out=z3)

tensor([[1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.]])

**Tensores de elemento único** Se você tiver um tensor de um elemento, 
por exemplo, agregando todos os valores de um tensor em um valor, 
você pode convertê-lo em um valor numérico Python usando `item()`:

In [43]:
agg = tensor.sum()
agg_item = agg.item()
print(agg_item, type(agg_item))

12.0 <class 'float'>


**Operações no local** operações que armazenam o resultado no execução 
são chamadas de operações no local. Elas são denotadas por um `_` sufixo. 
Por exemplo: `x.copy_(y)`, `x.t_()`, mudará `x`.

In [44]:
print(f"{tensor} \n")
tensor.add_(5)
print(tensor)

tensor([[1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.]]) 

tensor([[6., 5., 6., 6.],
        [6., 5., 6., 6.],
        [6., 5., 6., 6.],
        [6., 5., 6., 6.]])


> **Observação:**
> 
> Operações no local economizam memória, mas podem ser problemáticas ao calcular derivadas devido à perda imediata do histórico.  
> Portanto, seu uso é desencorajado.


------------------------------------------------------------------------


Ponte com NumPy
=================

Os tensores nas matrizes da CPU e do NumPy podem compartilhar seus locais de memória subjacentes, e alterar um alterará o outro.


Tensor para matriz NumPy
=====================


In [45]:
t = torch.ones(5)
print(f"t: {t}")
n = t.numpy()
print(f"n: {n}")

t: tensor([1., 1., 1., 1., 1.])
n: [1. 1. 1. 1. 1.]


Uma alteração no tensor é refletida na matriz NumPy.


In [46]:
t.add_(1)
print(f"t: {t}")
print(f"n: {n}")

t: tensor([2., 2., 2., 2., 2.])
n: [2. 2. 2. 2. 2.]


Matriz NumPy para Tensor
=====================


In [47]:
n = np.ones(5)
t = torch.from_numpy(n)

Alterações na matriz NumPy refletem no tensor.


In [48]:
np.add(n, 1, out=n)
print(f"t: {t}")
print(f"n: {n}")

t: tensor([2., 2., 2., 2., 2.], dtype=torch.float64)
n: [2. 2. 2. 2. 2.]
